In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import expipe
from expipe import Browser
import pandas as pd
import numpy as np
import sys

sys.path.append('../ca2-mec') if '../ca2-mec' not in sys.path else None 
import data_processing as dl
from utils import *

In [3]:
project = expipe.get_project(dl.project_path())
include_actions = ['011-120321-2', '011-120321-3', '011-120321-4', '011-120321-5', '011-120321-6'] # choose actions to include

/fp/homes01/u01/ec-bjornmik/.local/lib/python3.11/site-packages/expipe/core.py:700: UserWarning: Requested project with name 'mec_social', but found 'mec-social'
  warnings.warn(


### Create pandas dataframe layout

In [4]:
tot_time = ['unspecified']*len(include_actions)
top_right_time = ['unspecified']*len(include_actions)
bottom_left_time = ['unspecified']*len(include_actions)
sdi = ['unspecified']*len(include_actions)
BL = ['unspecified']*len(include_actions)
TL = ['unspecified']*len(include_actions)
TR = ['unspecified']*len(include_actions)
BR = ['unspecified']*len(include_actions)

data = {'action_id': include_actions, 'total-time-in-seconds': tot_time, \
        'time-spent-in-top-social-zone': top_right_time, 'time-spent-in-bottom-social-zone': bottom_left_time, \
        'social-discrimination-index': sdi, 'BL': BL, 'TL': TL, 'TR': TR, 'BR': BR}
df = pd.DataFrame(data)

### Populate pandas dataframe with social statistics

In [5]:
def process(lim=[0,1200],margin=0.4):
    def process_row(row):
        x,y,t,_ = dl.load_tracking(row['action_id'], lim=lim, ca2_transform_data=True).T

        social_types = social_label(row['action_id'])
        box_locations = ~(social_types== -1) # True where there is 'box', false on 'nobox'
        box_idx_masks = np.append(box_locations, False)
        cms = corner_masks(x=x, y=y, margin=margin) # bl,tl,tr,br,center respectively.
        sm1,sm2 = cms[:,box_idx_masks].T
        
        # Check if social mask 1 (sm1) is top corner. Make it top corner if not
        sm1_is_top_corner = np.argmax(box_locations) in [1,2] 
        top_sm, bottom_sm = (sm1,sm2) if sm1_is_top_corner else (sm2,sm1)

        total_recording_time = t[-1] - t[0]
        samp_freq = len(t) / total_recording_time
        top_time = np.sum(top_sm) / samp_freq
        bottom_time = np.sum(bottom_sm) / samp_freq
        sdi = (top_time - bottom_time) / (top_time + bottom_time)
        
        # Change SDI sign if familiar starts in the bottom of the box
        tags = project.require_action(row['action_id']).attributes["tags"]
        familiar_starts_in_top = 1
        for tag in tags:
            if tag.split('_')[0] == 'familiar' and tag.split('_')[-1][0] == 'b':
                familiar_starts_in_top = -1
        
        sdi = sdi * familiar_starts_in_top

        # Save values
        row['total-time-in-seconds'] = total_recording_time
        row['time-spent-in-top-social-zone'] = top_time
        row['time-spent-in-bottom-social-zone'] = bottom_time
        row['social-discrimination-index'] = sdi
        row['BL'] = social_types[0]
        row['TL'] = social_types[1]
        row['TR'] = social_types[2]
        row['BR'] = social_types[3]
        print(row)
        
    return process_row

### Save as expipe action csv

In [6]:
df.apply(process(lim=[0,1200],margin=0.4), axis=1)
dataframes = project.require_action('dataframes')
dataframes.data['sessions'] = 'sessions.csv'
df.to_csv(dataframes.data_path('sessions'), index=False)

UnsupportedOperation: Unable to open file /projects/ec109/maria/mec_social/actions/011-120321-2/data/main.exdir in 'r' mode. File does not exist.